# Dev notebook for manual work

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Creating ptr.csv

In [3]:
# read excel file in data
data = pd.read_excel('../../data/Additional file 2.xlsx', sheet_name='PTR_Eraslan2019')

In [5]:
# Store
data.to_csv('../../data/ptr.csv', index=False)

In [4]:
data.head(10)

,GeneName,EnsemblGeneID,EnsemblTranscriptID,EnsemblProteinID,Adrenal,Appendices,Brain,Colon,Duodenum,Uterus,...,Rectum,Salivarygland,Smallintestine,Smoothmuscle,Spleen,Stomach,Testis,Thyroid,Tonsil,Urinarybladder
0,A1BG,ENSG00000121410,ENST00000263100,ENSP00000263100,NaN,8.277,NaN,NaN,NaN,NaN,...,NaN,7.718,NaN,NaN,7.313,NaN,NaN,NaN,NaN,NaN
1,A1CF,ENSG00000148584,ENST00000373993,ENSP00000363105,NaN,NaN,NaN,5.135,5.371,NaN,...,5.147,NaN,5.202,NaN,NaN,5.8143,NaN,NaN,NaN,NaN
2,A2M,ENSG00000175899,ENST00000318602,ENSP00000323929,6.290,6.328,5.948,5.811,6.068,5.383,...,6.081,5.726,5.699,4.997,5.136,6.5349,5.820,6.060,5.675,5.8286
3,A2ML1,ENSG00000166535,ENST00000299698,ENSP00000299698,NaN,NaN,3.995,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,2.350,NaN,5.249,NaN
4,A4GALT,ENSG00000128274,ENST00000401850,ENSP00000384794,3.843,4.601,NaN,NaN,NaN,NaN,...,4.731,4.508,NaN,NaN,NaN,4.0613,4.832,NaN,NaN,4.2430
5,A4GNT,ENSG00000118017,ENST00000236709,ENSP00000236709,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.3992,NaN,NaN,NaN,NaN
6,AAAS,ENSG00000094914,ENST00000209873,ENSP00000209873,4.513,4.434,4.461,4.536,4.917,4.677,...,4.440,4.350,4.120,4.241,3.975,5.0890,4.991,4.651,4.505,4.4699
7,AACS,ENSG00000081760,ENST00000316519,ENSP00000324842,4.559,5.015,5.120,4.426,5.285,4.640,...,4.817,4.807,4.395,4.310,NaN,4.7653,4.552,4.588,4.901,4.8409
8,AADAC,ENSG00000114771,ENST00000232892,ENSP00000232892,5.110,NaN,NaN,NaN,5.008,NaN,...,NaN,NaN,5.076,NaN,NaN,4.7098,NaN,NaN,NaN,NaN
9,AADAT,ENSG00000109576,ENST00000515480,ENSP00000423341,5.227,NaN,4.165,4.115,NaN,4.801,...,4.282,NaN,NaN,NaN,NaN,4.5203,3.657,3.563,NaN,4.2271


## Loading dev data file

In [7]:
# load data.pkl
import pickle

with open('../../data/ptr_data.pkl', 'rb') as f:
    data = pickle.load(f)
    
# Insights:
# - data is a dictionary with keys as gene ids
# - targets are tissue specific PTRs
# - targets_bin 1 if "up", 0 if "down"

In [9]:
len(data.keys())

11279

In [10]:
data["ENST00000012049"].keys()

dict_keys(['fasta', 'fasta_ohe', 'bed', 'bed_annotation', 'codon_freq', 'targets', 'targets_bin'])

In [11]:
data["ENST00000012049"]

{'fasta': 'AATCCGTGGTCTGGTACAGGTTTCAGGGCAAAGCGGCCATGCGTTCCGGGGGCCGCGGGCGACCCCGCCTGCGGCTGGGGGAACGTGGCCTCATGGAGCCACTCTTGCCGCCGAAGCGCCGCCTGCTACCGCGGGTTCGGCTCTTGCCTCTGTTGCTGGCGCTGGCCGTGGGCTCGGCGTTCTACACCATTTGGAGCGGCTGGCACCGCAGGACTGAGGAGCTGCCGCTGGGCCGGGAGCTGCGGGTCCCATTGATCGGAAGCCTCCCCGAAGCCCGGCTGCGGAGGGTGGTGGGACAACTGGATCCACAGCGTCTCTGGAGCACTTATCTGCGCCCCCTGCTGGTTGTGCGAACCCCGGGCAGCCCGGGAAATCTCCAAGTCAGAAAGTTCCTGGAGGCCACGCTGCGGTCCCTGACAGCAGGTTGGCACGTGGAGCTGGATCCCTTCACAGCCTCAACACCCCTGGGGCCAGTGGACTTTGGCAATGTGGTGGCCACACTGGACCCAAGGGCTGCCCGTCACCTCACCCTTGCCTGCCATTATGACTCGAAGCTCTTCCCACCCGGATCGACCCCCTTTGTAGGGGCCACGGATTCGGCTGTGCCCTGTGCCCTGCTGCTGGAGCTGGCCCAAGCACTTGACCTGGAGCTGAGCAGGGCCAAAAAACAGGCAGCCCCGGTGACCCTGCAACTGCTCTTCTTGGATGGTGAAGAGGCGCTGAAGGAGTGGGGACCCAAGGACTCCCTTTACGGTTCCCGGCACCTGGCCCAGCTCATGGAGTCTATACCTCACAGCCCCGGCCCCACCAGGATCCAGGCTATTGAGCTCTTTATGCTTCTTGATCTCCTGGGAGCCCCCAATCCCACCTTCTACAGCCACTTCCCTCGCACGGTCCGCTGGTTCCATCGGCTGAGGAGCATTGAGAAGCGTCTGCACCGTTTGAACCTGCTGCAGTCTCATCCCCAGGAAGTGATGTACTTCCAACCC

In [14]:
len(data["ENST00000012049"]["bed_annotation"])

2125

In [13]:
len(data["ENST00000012049"]["fasta_ohe"])

2125